## Back of the envelope performance analysis

We might expect coppertop to be slower than regular Python - every function call has to got through a complex dispatch 
process. It is slower. Here we explain where it is slower, how to mitigate that when it is an issue and even why that 
can be an advantage.

<br>


Firstly, some philosophical remarks from some old colleagues:

Colin, an old colleague once said, Smalltalk is faster than other languages because I can write the code 5 times quicker,
which leaves me 80% of the time to analyse it and make it faster. Smalltalk could call directly into the C-ABI which 
we used on a couple of occasions.

To paraphrase another colleague, Neil - You have heard it said that "Premature optimisation is the root of all evil" but
I say unto you that there are two forms of optimisation, course-grained and fine-grained, and fine we can leave til the 
end and only do if necessary.

Fine-grained is about loops, tricks and dirty code to speed up identified bottle-necks (Smalltalk-80 had performance 
profiling tools back in the late 80s, early 90s and maybe earlier). It is fun to do, compelling even, but in general 
unnecessary, a waste of time, and destructive to code clarity.

Course-grained is the optimisation that is done at an architectural level. Being architectural it must be done upfront 
to avoid unnecessary rework.

See - https://dl.acm.org/ft_gateway.cfm?id=1513451&type=pdf - for a fuller discussion.


<br>

Below we take an example I needed a couple of years ago - partitioning a list into groups in all combinations. The algo
has been solved on Rosetta Code - see https://rosettacode.org/wiki/Ordered_partitions#Python.

We compare the naive Python implementation with an ML style one and several coppertop ones.

Our presenting problem is creating the 90090 combinations from a list of 13 partitioned into groups of 5, 4 and 4 elements - 
all the possible starting hands of opponents in a four player game of Cludeo. I never had the patience to let the naive 
version run its course.

<br>

CONTENTS

1 - brute force filtering itertool's permuations output \
2 - ML inspired \
3a - fred \
3b - 

In [4]:
import itertools

from coppertop.pipe import *
from dm.core.types import pylist, index
from dm.testing import check, equals
from dm.core import first, count, drop, collect, prependTo, join, joinAll, take, sum, unpack
from dm.pp import PP

BRUTE FORCE

In [5]:
def partitionsBruteForce(cards, handSizes):
    slices = []
    s1 = 0
    for handSize in handSizes:
        s2 = s1 + handSize
        slices.append((s1, s2))
        s1 = s2
    perms = filter(
        lambda perm: groupsInOrder(perm, slices),
        itertools.permutations(cards, len(cards))
    )
    return tuple(perms)

def groupsInOrder(xs, slices):
    for s1, s2 in slices:
        if not isAsc(xs[s1:s2]): return False
    return True

def isAsc(xs):
    p = xs[0]
    for n in xs[1:]:
        if n <= p: return False
        p = n
    return True

In [6]:
partitionsBruteForce((1,2,3,4,5), (2,3))

((1, 2, 3, 4, 5),
 (1, 3, 2, 4, 5),
 (1, 4, 2, 3, 5),
 (1, 5, 2, 3, 4),
 (2, 3, 1, 4, 5),
 (2, 4, 1, 3, 5),
 (2, 5, 1, 3, 4),
 (3, 4, 1, 2, 5),
 (3, 5, 1, 2, 4),
 (4, 5, 1, 2, 3))

In [13]:
partitionsBruteForce(list(range(10)), [4,3,3]) >> count

4200

In [12]:
%timeit  partitionsBruteForce(list(range(10)), [4,3,3]) >> count

909 ms ± 5.31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


ML-STYLE

In [4]:
# partitions :: [Int] -> [[[Int]]]
def partitionsML(xs, sizes):
    n = sum(sizes)

    def go(xs, n, sizes):
        return [
            [l] + r
            for (l, rest) in choose(xs)(n)(sizes[0])
            for r in go(rest, n - sizes[0], sizes[1:])
        ] if sizes else [[]]

    return go(xs, n, sizes)


# choose :: [Int] -> Int -> Int -> [([Int], [Int])]
def choose(xs):
    '''(m items chosen from n items, the rest)'''

    def go(xs, n, m):
        f = cons(xs[0])
        choice = choose(xs[1:])(n - 1)
        return [([], xs)] if 0 == m else (
            [(xs, [])] if n == m else (
                    [first(f)(xy) for xy in choice(m - 1)] +
                    [second(f)(xy) for xy in choice(m)]
            )
        )

    return lambda n: lambda m: go(xs, n, m)


# cons :: a -> [a] -> [a]
def cons(x):
    '''Construction of a list from x as head, and xs as tail.'''
    return lambda xs: [x] + xs


# first :: (a -> b) -> ((a, c) -> (b, c))
def first(f):
    '''A simple function lifted to a function over a tuple, with f applied only the first of two values.'''
    return lambda xy: (f(xy[0]), xy[1])


# second :: (a -> b) -> ((c, a) -> (c, b))
def second(f):
    '''A simple function lifted to a function over a tuple, with f applied only the second of two values.'''
    return lambda xy: (xy[0], f(xy[1]))

In [5]:
partitionsML([1,2,3,4,5], (2,3))

[[[1, 2], [3, 4, 5]],
 [[1, 3], [2, 4, 5]],
 [[1, 4], [2, 3, 5]],
 [[1, 5], [2, 3, 4]],
 [[2, 3], [1, 4, 5]],
 [[2, 4], [1, 3, 5]],
 [[2, 5], [1, 3, 4]],
 [[3, 4], [1, 2, 5]],
 [[3, 5], [1, 2, 4]],
 [[4, 5], [1, 2, 3]]]

In [6]:
%timeit  partitionsML(list(range(13)), [5,4,4]) >> count >> PP

90090
90090
90090
90090
90090
90090
90090
90090
331 ms ± 2.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


COPPERTOP-V1

In [7]:
from dm.core import first

In [8]:
@coppertop(style=binary)
def takeDrop(xs, s):
    return xs[:s], xs[s:]

In [9]:
@coppertop(style=binary)
def partitionsCpt(xs, sizes: pylist) -> pylist:
    sizes >> sum >> check >> equals >> (xs >> count)
    return _partitionsCpt(list(xs), xs >> count, sizes)

@coppertop
def _partitionsCpt(xs: pylist, n: index, sizes: pylist) -> pylist:
    if sizes:
        return xs >> _combRestCpt(_, n, sizes >> first) \
            >> collect >> (unpack(lambda x, y:
                _partitionsCpt(y, n - (sizes >> first), sizes >> drop >> 1)
                    >> collect >> (lambda partitions:
                        x >> prependTo >> partitions
                    )
            )) \
            >> joinAll
    else:
        return [[]]

@coppertop
def _combRestCpt(xs: pylist, n: index, m: index) -> pylist:
    '''answer [m items chosen from n items, the rest]'''
    if m == 0: 
        return [([], xs)]
    elif m == n: 
        return [(xs, [])]
    else:
        s1, s2 = xs >> takeDrop >> 1
        return \
            (s2 >> _combRestCpt(_, n - 1, m - 1) >> collect >> unpack(lambda x, y: (s1 >> join >> x, y))) \
            >> join >> \
            (s2 >> _combRestCpt(_, n - 1, m) >> collect >> unpack(lambda x, y: (x, s1 >> join >> y)))

In [10]:
[1,2,3,4,5] >> partitionsCpt >> [2,3]

[[[1, 2], [3, 4, 5]],
 [[1, 3], [2, 4, 5]],
 [[1, 4], [2, 3, 5]],
 [[1, 5], [2, 3, 4]],
 [[2, 3], [1, 4, 5]],
 [[2, 4], [1, 3, 5]],
 [[2, 5], [1, 3, 4]],
 [[3, 4], [1, 2, 5]],
 [[3, 5], [1, 2, 4]],
 [[4, 5], [1, 2, 3]]]

In [11]:
%timeit range(13) >> partitionsCpt >> [5,4,4] >> count >> PP

90090
90090
90090
90090
90090
90090
90090
90090
6.86 s ± 84.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


COPPERTOP-V2

In [12]:
@coppertop(style=binary)
def partitionsCptFaster(xs, sizes: pylist) -> pylist:
    sizes >> sum >> check >> equals >> (xs >> count)
    return _partitionsCptFaster(list(xs), xs >> count, sizes)

@coppertop
def _partitionsCptFaster(xs: pylist, n: index, sizes: pylist) -> pylist:
    if not sizes: return [[]]
    return _combRestCptFaster(xs, n, sizes[0]) \
        >> collect >> (unpack(lambda comb, rest:
            _partitionsCptFaster(rest, n - sizes[0], sizes[1:])
                >> collect >> (lambda partitions:
                    [comb] + partitions
                )
        )) \
        >> joinAll

@coppertop
def _combRestCptFaster(xs: pylist, n: index, m: index) -> pylist:
    '''answer [m items chosen from n items, the rest]'''
    if m == 0: return [([], xs)]
    if m == n: return [(xs, [])]
    s1, s2 = xs[:1], xs[1:]
    return \
        (_combRestCptFaster(s2, n - 1, m - 1) >> collect >> (lambda xy: (s1 + xy[0], xy[1]))) + \
        (_combRestCptFaster(s2, n - 1, m) >> collect >> (lambda xy: (xy[0], s1 + xy[1])))


In [13]:
[1,2,3,4,5] >> partitionsCptFaster >> [2,3]

[[[1, 2], [3, 4, 5]],
 [[1, 3], [2, 4, 5]],
 [[1, 4], [2, 3, 5]],
 [[1, 5], [2, 3, 4]],
 [[2, 3], [1, 4, 5]],
 [[2, 4], [1, 3, 5]],
 [[2, 5], [1, 3, 4]],
 [[3, 4], [1, 2, 5]],
 [[3, 5], [1, 2, 4]],
 [[4, 5], [1, 2, 3]]]

In [14]:
%timeit range(13) >> partitionsCptFaster >> [5,4,4] >> count >> PP

90090
90090
90090
90090
90090
90090
90090
90090
3.33 s ± 22.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


COPPERTOP-V3

In [15]:
@coppertop(style=binary)
def partitionsCptFasterStill(xs, sizes: pylist) -> pylist:
    sizes >> sum >> check >> equals >> (xs >> count)
    return _partitionsCptFasterStill(list(xs), xs >> count, sizes)

def _partitionsCptFasterStill(xs: pylist, n: index, sizes: pylist) -> pylist:
    if not sizes: return [[]]
    return list(itertools.chain(*map(
        lambda comb_rest: map(
            lambda partitions: [comb_rest[0]] + partitions,
            _partitionsCptFasterStill(comb_rest[1], n - sizes[0], sizes[1:])
        ),
        _combRestCptFasterStill(xs, n, sizes[0])
    )))

def _combRestCptFasterStill(xs: pylist, n: index, m: index) -> pylist:
    '''answer [m items chosen from n items, the rest]'''
    if m == 0: return [([], xs)]
    if m == n: return [(xs, [])]
    s1, s2 = xs[:1], xs[1:]
    return list(itertools.chain(
        map(lambda xy: (s1 + xy[0], xy[1]), _combRestCptFasterStill(s2, n - 1, m - 1)),
        map(lambda xy: (xy[0], s1 + xy[1]), _combRestCptFasterStill(s2, n - 1, m))
    ))

In [16]:
[1,2,3,4,5] >> partitionsCptFasterStill >> [2,3]

[[[1, 2], [3, 4, 5]],
 [[1, 3], [2, 4, 5]],
 [[1, 4], [2, 3, 5]],
 [[1, 5], [2, 3, 4]],
 [[2, 3], [1, 4, 5]],
 [[2, 4], [1, 3, 5]],
 [[2, 5], [1, 3, 4]],
 [[3, 4], [1, 2, 5]],
 [[3, 5], [1, 2, 4]],
 [[4, 5], [1, 2, 3]]]

In [17]:
%timeit range(13) >> partitionsCptFasterStill >> [5,4,4] >> count >> PP

90090
90090
90090
90090
90090
90090
90090
90090
202 ms ± 4.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


From a pytest run:
```
test_cp_1.py took 3581.21575 ms
test_cp_2.py took 1793.398834 ms
test_cp_3.py took 490.383291 ms
test_cp_4.py took 3597.319334 ms
test_cp_5.py took 1266.624375 ms
test_cp_6.py took 677.777916 ms
test_cp_7.py took 6319.097166 ms
test_ml_style.py took 401.368209 ms
```


BONES

```
load dm.core
from dm.core import sum, count, isEmpty, ifTrue:, collect, first, drop, prependTo, joinAll, takeDrop, join
from dm.testing import check, equal

partitions: {{[xs:N1**T1, sizes:N2**count] <:N**N2**N**T1>
    sizes sum check equal (xs count)
    xs _partitions(, xs count, sizes)
}}

_partitions: {[xs:N1**T1, n:count, sizes:N2**count] <:N**N2**N**T1>
    sizes isEmpty ifTrue: [^ (())]
    xs _combRest(, n, sizes first) collect {[a, b]
        _partitions(b, .n - (.sizes first), .sizes drop 1) collect {
            .a prependTo r
        }
    } joinAll
}

_combRest: {[xs:N**T1, n:count, m:count] <:N**(N**T1)*(N**T1)>
    m == 0 ifTrue: [^ ( (() <:N**T1>, xs) )]
    m == n ifTrue: [^ ( (xs, () <:N**T1>) )]
    (comb, rest): xs takeDrop 1
    rest _combRest(, s2 count, m - 1) collect { (.comb join a, b) }    // #1
      join
      (rest _combRest(, s2 count, m) collect { (a, .comb join b) })    // #2
}
```

CONCLUSION

It is hard writing fast code in python for this problem (one range / itertool based solution was 16x slower than the base ML style!

Removing dispatch and type checking - coppertop is 1.7x slower than the ML version.

My fastest itertools version is, depending on arguments, 30% faster to 20% slower than the list based ML version.

My unoptimised coppertop version is 8x slower than the ML version.





WHY?

Coppertop dispatch is written in Python. At its best the C piping code has to the overhead of the pipe plus the 
call into the free function. Nullary, unary and binary functions called in Fortran style should be called as fast 
as a regular python function. However, additional function calls is required for missing argument.

```
1 arg
f(x)            - 1 call
x >> f          - 1 call

2 args
f(x, y)         - 1 call
x >> f(_, y)    - 2 calls
x >> f >> y     - 2 calls

3 args
f(x, y, z)                          - 1 call
x >> f(_, y, z)                     - 2 calls
x >> f(_, _, z) >> y                - 3 calls
x >> f >> y >> z                    - 3 calls

4 or more args
x >> f(_, _, _, fred) >> y >> z     - 4 calls
```

calling methods is slower than calling free functions -  performance to be analysed

lambdas are 2-3x slower than free functions

As the dispatch is moved into C there will be less need to remove type based dispatch for tight loops.

Coppertop can't go faster than Python however it is competitive with lambdas, closures and classes. However it 
is much more productive to code with.

